In [1]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

from pyAgrum.lib._utils.oslike import head

### generating data with missing values (at random)

In [2]:
src=gum.fastBN("A->B<-C->D->E<-B;D->F")
gum.generateCSV(src,"EM_nomissing.csv",5000,random_order=False)
src

(gum::BayesNet<double>@0x55e603aefe80) BN{nodes: 6, arcs: 6, domainSize: 64, parameters: 28, compression ratio: 56% }

In [3]:
import pandas as pd
import numpy as np

def add_missing(src,dst,proba):
  df=pd.read_csv(src)
  mask=np.random.choice([True, False], size=df.shape,p=[proba,1-proba])
  df.mask(mask).to_csv(dst,na_rep='?',index=False,float_format='%.0f')

gum.generateCSV(src,"EM_nomissing.csv",5000,random_order=False)
add_missing("EM_nomissing.csv","EM_missing.csv",proba=0.1)

In [4]:
print("No missng")
head("EM_nomissing.csv")
print("Missing")
head("EM_missing.csv")

No missng
A,B,C,D,E,F
1,0,1,0,1,1
0,1,1,1,0,0
1,0,0,1,0,0
1,1,0,0,1,1
1,0,0,0,1,0
1,0,1,0,1,0
1,0,0,1,1,0
1,1,1,1,1,0
1,0,0,1,0,0

Missing
A,B,C,D,E,F
1,?,1,0,1,1
0,1,?,1,0,0
1,0,0,1,0,?
1,1,0,0,1,1
1,0,0,0,1,0
1,0,1,0,1,0
1,0,0,1,1,0
?,?,1,1,1,0
1,0,?,1,0,0



### learning with missing data

In [5]:
learner = gum.BNLearner('EM_missing.csv', ["?"])
print(f"Missing values in EM_missing.csv : {learner.hasMissingValues()}")

Missing values in EM_missing.csv : True


In [6]:
# this will fail : missing data !
# learner.learnParameters(src.dag())

In [7]:
learner.useEM(1e-3)
learner.useAprioriSmoothing()
bn=learner.learnParameters(src.dag())
print(f"# iterations : {learner.nbrIterations()}")
gnb.sideBySide(gnb.getInference(src),gnb.getInference(bn))

# iterations : 4


,


### learning with smaller error (and no smoothing)

In [8]:
learner = gum.BNLearner('EM_missing.csv', ["?"])
learner.setVerbosity(True)
learner.useEM(1e-8)
bn2=learner.learnParameters(src.dag())
print(f"# iterations : {learner.nbrIterations()}")
gnb.sideBySide(gnb.getInference(src),gnb.getInference(bn2))

# iterations : 13


,


In [9]:
print(learner.history())

(0.11253761295326103, 0.011407420780116336, 0.0025144028695680593, 0.0005868706478227338, 0.00014240793154775508, 3.562349270626642e-05, 9.129729192605464e-06, 2.3843169498453966e-06, 6.316145671441013e-07, 1.6907902826300825e-07, 4.5604552203744306e-08, 1.2366809252880485e-08, 3.3662463735503003e-09)
